In [22]:
import torch
import numpy as np
from torch import nn
import tensorly as tl
from tqdm import tqdm
import matplotlib.pyplot
from torch.optim import Adam
from torch.utils.data import DataLoader

In [2]:
from tensor_layer import *

In [21]:
tl.set_backend('pytorch')

In [19]:
train_set = MOA_set("training_X.npy", "training_Y.npy")
test_set = MOA_set("testing_X.npy")

In [24]:
training_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

In [ ]:
loss = 

In [ ]:
num_iter = 1000
batch_size = 100
errors = []
batch_count = len(data) // batch_size if len(data) % batch_size == 0 else len(data) // batch_size + 1

for i in tqdm(range(num_iter)):
    
    mean_loss = 0
    
    for batch in range(batch_count):
        
        inp = torch.Tensor(data[batch * batch_size:(batch + 1) * batch_size])
        tar = torch.Tensor(target[batch * batch_size:(batch + 1) * batch_size]).type(torch.long)
    
        optimizer.zero_grad()
        output = NN(inp)
        loss_val = loss(output, torch.flatten(tar))
        loss_val.backward()
        optimizer.step()
        
        mean_loss += loss_val
        
    errors.append(mean_loss / batch_count)
        
    if i % 100 == 0:
        print(loss_val)